# FAF5 Network Analysis with AequilibraE

This notebook demonstrates how to:
1. Load the FAF5 network from Geodatabase format
2. Create an AequilibraE project and import the network
3. Calculate shortest travel times/costs between OD-pairs
4. Compute shortest paths between OD-pairs
5. Generate route choice sets between OD pairs
6. Perform traffic assignment using path-sized logit method


In [40]:
# Imports
import os
import geopandas as gpd
import pandas as pd
import numpy as np
from uuid import uuid4
from tempfile import gettempdir
from os.path import join
from shapely.geometry import Point, LineString
from shapely.wkb import dumps as wkb_dumps
from shapely.strtree import STRtree
from shapely.ops import linemerge

from aequilibrae import Project
from aequilibrae.paths import RouteChoice
from aequilibrae.paths.traffic_assignment import TrafficAssignment
from aequilibrae.paths.traffic_class import TrafficClass
from aequilibrae.matrix import AequilibraeMatrix
from aequilibrae.project.project_creation import remove_triggers, add_triggers

print("Libraries imported successfully")


Libraries imported successfully


## 1. Load FAF5 Network Data

Load the FAF5 network layers from the Geodatabase format.


In [41]:
# Path to the FAF5 Geodatabase
gdb_path = "Networks/Geodatabase Format/FAF5Network.gdb"

# Load FAF5 layers
print("Loading FAF5 network layers...")
links_gdf = gpd.read_file(gdb_path, layer="FAF5_Links")
nodes_gdf = gpd.read_file(gdb_path, layer="FAF5_Nodes")

print(f"Loaded {len(links_gdf)} links and {len(nodes_gdf)} nodes")
print(f"\nLinks columns: {list(links_gdf.columns)}")
print(f"\nNodes columns: {list(nodes_gdf.columns)}")
print(f"\nLinks head:")
links_gdf.head()


Loading FAF5 network layers...
Loaded 487394 links and 974788 nodes

Links columns: ['ID', 'LENGTH', 'DIR', 'DATA', 'VERSION', 'Class', 'Class_Description', 'Road_Name', 'Sign_Rte', 'Rte_Type', 'Rte_Number', 'Rte_Qualifier', 'Country', 'STATE', 'STFIPS', 'County_Name', 'CTFIPS', 'Urban_Code', 'FAFZONE', 'Status', 'F_Class', 'Facility_Type', 'NHS', 'STRAHNET', 'NHFN', 'Truck', 'AB_Lanes', 'BA_Lanes', 'Speed_Limit', 'Toll_Type', 'Toll_Name', 'Toll_Link', 'Toll_Link_Name', 'HPMS_USA_RouteID', 'HPMS_Begin_Point', 'HPMS_End_Point', 'BorderState1', 'BorderState2', 'BorderFAF1', 'BorderFAF2', 'TRUCKTOLL', 'BorderLink', 'AddedBorderTime', 'AdjustSpeed', 'AdjustReason', 'AB_FinalSpeed', 'BA_FinalSpeed', 'AB_CombinedSpeed', 'BA_CombinedSpeed', 'AB_FreeFlowTime', 'BA_FreeFlowTime', 'SHAPE_Length', 'geometry']

Nodes columns: ['ID', 'DATA', 'Entry_or_Exit', 'Exit_Number', 'Interchange', 'Centroid', 'CentroidID', 'Facility_Type', 'Facility_Name', 'County', 'State', 'StateID', 'StateName', 'FAFID', 

,ID,LENGTH,DIR,DATA,VERSION,Class,Class_Description,Road_Name,Sign_Rte,Rte_Type,...,AdjustSpeed,AdjustReason,AB_FinalSpeed,BA_FinalSpeed,AB_CombinedSpeed,BA_CombinedSpeed,AB_FreeFlowTime,BA_FreeFlowTime,SHAPE_Length,geometry
0,1,0.054083,1,1324805,V2021.05,19.0,Facility Access/Circulator Road,PETERSBURG FERRY TERMINAL RD,None,None,...,NaN,None,43.0,43.0,43.0,43.0,0.075464,0.075464,0.001009,"MULTILINESTRING ((-132.97501 56.80636, -132.97..."
1,2,0.114854,0,1324806,V2021.05,19.0,Facility Access/Circulator Road,PETERSBURG FERRY TERMINAL RD,None,None,...,NaN,None,43.0,43.0,43.0,43.0,0.160261,0.160261,0.002005,"MULTILINESTRING ((-132.97503 56.80701, -132.97..."
2,3,0.032432,1,1324807,V2021.05,19.0,Facility Access/Circulator Road,PETERSBURG FERRY TERMINAL RD,None,None,...,NaN,None,43.0,43.0,43.0,43.0,0.045254,0.045254,0.000626,"MULTILINESTRING ((-132.97503 56.80701, -132.97..."
3,4,0.626287,0,126521,V2021.05,14.0,Arterial or Major Collector,NORDIC DR,None,None,...,NaN,None,28.0,28.0,28.0,28.0,1.342044,1.342044,0.015130,"MULTILINESTRING ((-132.97455 56.80664, -132.97..."
4,5,15.223004,0,1324808,V2021.05,41.0,Ferry,WRANGELL-PETERSBURG FERRY,None,None,...,NaN,None,10.0,10.0,10.0,10.0,91.338026,91.338026,0.344370,"MULTILINESTRING ((-132.97615 56.80849, -132.97..."


In [42]:
# Display nodes head
print("Nodes head:")
nodes_gdf.head()


Nodes head:


,ID,DATA,Entry_or_Exit,Exit_Number,Interchange,Centroid,CentroidID,Facility_Type,Facility_Name,County,State,StateID,StateName,FAFID,StateNameBak,geometry
0,574,47154824,None,None,None,NaN,NaN,None,None,None,None,NaN,None,NaN,None,POINT (-132.97501 56.80636)
1,2,47154826,None,None,None,NaN,NaN,None,None,None,None,NaN,None,NaN,None,POINT (-132.97503 56.80701)
2,2,47154826,None,None,None,NaN,NaN,None,None,None,None,NaN,None,NaN,None,POINT (-132.97503 56.80701)
3,3,47154825,None,None,None,NaN,NaN,None,None,None,None,NaN,None,NaN,None,POINT (-132.97615 56.80849)
4,2,47154826,None,None,None,NaN,NaN,None,None,None,None,NaN,None,NaN,None,POINT (-132.97503 56.80701)


## 2. Create AequilibraE Project

Create a new AequilibraE project and import the FAF5 network data.


In [43]:
# Create a new AequilibraE project
project_folder = join(gettempdir(), f"faf5_analysis_{uuid4().hex[:8]}")
project = Project()
project.new(project_folder)
print(f"Created project at: {project_folder}")


No pre-existing parameter file exists for this project. Will use default
No pre-existing parameter file exists for this project. Will use default


Created project at: /var/folders/1b/ybckpz0d0d19bczw9mwmf_r40000gn/T/faf5_analysis_fe1733db


In [44]:
# Add custom fields to links table if needed (before bulk insert)
print("Preparing custom fields...")
links = project.network.links
link_fields = links.fields

# Add fields for FAF5 data
try:
    link_fields.add("travel_time", "Free flow travel time", "REAL")
    link_fields.add("speed", "Speed limit", "REAL")
    link_fields.add("source_id", "Original FAF5 link ID", "INTEGER")
    links.refresh_fields()
except:
    pass  # Fields may already exist

print("Custom fields prepared")


Preparing custom fields...
Custom fields prepared


In [45]:
# Bulk insert nodes using SQL for performance
print("Preparing nodes for bulk insertion...")

# Prepare node data: extract coordinates and map fields
node_data = []
node_coords = {}  # Store for link creation
seen_node_ids = set()  # Track node IDs to avoid duplicates
duplicate_count = 0

for idx, row in nodes_gdf.iterrows():
    node_id = int(row['ID'])
    
    # Skip if we've already seen this node_id
    if node_id in seen_node_ids:
        duplicate_count += 1
        continue
    
    seen_node_ids.add(node_id)
    is_centroid = 1 if row.get('Centroid', 0) == 1 else 0
    
    # Extract coordinates from geometry
    if row.geometry is not None and row.geometry.geom_type == 'Point':
        lon = row.geometry.x
        lat = row.geometry.y
        node_coords[node_id] = (lon, lat)
        # Convert Point geometry to WKB for efficient insertion
        geom_wkb = wkb_dumps(row.geometry, hex=False)
    else:
        # Fallback: try to get x, y attributes
        try:
            lon = row.geometry.x
            lat = row.geometry.y
            node_coords[node_id] = (lon, lat)
            # Create Point from coordinates if geometry is not a Point
            point = Point(lon, lat)
            geom_wkb = wkb_dumps(point, hex=False)
        except:
            print(f"Warning: Could not extract coordinates for node {node_id}")
            continue
    
    # Prepare tuple for SQL insert: (node_id, is_centroid, modes, link_types, geometry_wkb)
    node_data.append((node_id, is_centroid, '', '', geom_wkb))

if duplicate_count > 0:
    print(f"Warning: Found {duplicate_count} duplicate node IDs, keeping first occurrence")
print(f"Prepared {len(node_data)} unique nodes for insertion")

# Bulk insert nodes
print("Bulk inserting nodes...")
with project.db_connection_spatial as conn:
    conn.execute("PRAGMA foreign_keys = ON")
    
    # Remove triggers for faster insertion
    remove_triggers(conn, project.logger, "network")
    
    # Insert nodes using GeomFromWKB for geometry (same approach as links)
    # Use INSERT OR IGNORE to handle any remaining duplicates gracefully
    insert_qry = """INSERT OR IGNORE INTO nodes (node_id, is_centroid, modes, link_types, geometry) 
                    VALUES(?, ?, ?, ?, GeomFromWKB(?, 4326))"""
    conn.executemany(insert_qry, node_data)
    conn.commit()
    
    # Re-add triggers
    add_triggers(conn, project.logger, "network")
    conn.commit()

print(f"Successfully inserted {len(node_data)} nodes")


Preparing nodes for bulk insertion...
Prepared 348495 unique nodes for insertion
Bulk inserting nodes...
Successfully inserted 348495 nodes


In [ ]:
# Bulk insert links using SQL for performance
# FAF5 links have DIR field: 0 = bidirectional, 1 = AB only
print("Preparing links for bulk insertion...")

# First, get the actual node IDs that were successfully inserted into the database
print("Querying database for inserted nodes...")
with project.db_connection_spatial as conn:
    result = conn.execute("SELECT node_id, X(geometry) as lon, Y(geometry) as lat FROM nodes")
    db_nodes = {row[0]: (row[1], row[2]) for row in result.fetchall()}

print(f"Found {len(db_nodes)} nodes in database")

# Filter node_coords to only include nodes that exist in the database
valid_node_coords = {node_id: coords for node_id, coords in node_coords.items() if node_id in db_nodes}
print(f"Using {len(valid_node_coords)} valid nodes for link matching")

# Create spatial index for faster node matching using only valid nodes
node_points = [Point(coord[0], coord[1]) for coord in valid_node_coords.values()]
node_tree = STRtree(node_points)
node_ids_list = list(valid_node_coords.keys())

# Helper function to find nearest node
def find_nearest_node(point, max_distance=0.01):
    """Find nearest node to a point within max_distance (degrees)"""
    if point is None:
        return None
    # Query spatial index
    candidates = node_tree.query(point.buffer(max_distance))
    if len(candidates) == 0:
        return None
    # Find closest
    distances = [point.distance(node_points[i]) for i in candidates]
    nearest_idx = candidates[np.argmin(distances)]
    return int(node_ids_list[nearest_idx])

# Prepare link data for bulk insertion
link_data = []
link_id_counter = 1

for idx, row in links_gdf.iterrows():
    source_link_id = int(row['ID'])
    direction = int(row.get('DIR', 0))  # 0 = bidirectional, 1 = AB only
    geom = row.geometry
    
    # Extract endpoints from geometry and convert MultiLineString to LineString
    if geom is None:
        continue
    
    # Convert MultiLineString to LineString if needed (required by AequilibraE)
    if geom.geom_type == 'MultiLineString':
        # Try to merge the MultiLineString into a single LineString
        try:
            merged = linemerge(geom)
            if merged.geom_type == 'LineString':
                geom = merged
            else:
                # If merge failed, use the first LineString segment
                geom = geom.geoms[0]
        except:
            # If merge fails, use the first LineString segment
            geom = geom.geoms[0]
    
    if geom.geom_type == 'LineString':
        coords = list(geom.coords)
        start_point = Point(coords[0])
        end_point = Point(coords[-1])
    else:
        # Skip if still not a LineString after conversion
        continue
    
    # Find nearest nodes to endpoints
    a_node_id = find_nearest_node(start_point)
    b_node_id = find_nearest_node(end_point)
    
    # Validate that both nodes exist in the database
    if a_node_id is None or b_node_id is None:
        continue
    if a_node_id not in db_nodes or b_node_id not in db_nodes:
        continue
    
    # Get link attributes
    travel_time_ab = row.get('AB_FreeFlowTime', np.nan)
    travel_time_ba = row.get('BA_FreeFlowTime', np.nan)
    length = row.get('LENGTH', 0.0)
    if pd.isna(length):
        length = 0.0
    
    # Get capacity (lanes * 1000 as default capacity per lane)
    capacity_ab = row.get('AB_Lanes', 1.0) * 1000 if not pd.isna(row.get('AB_Lanes', np.nan)) else 1000
    capacity_ba = row.get('BA_Lanes', 1.0) * 1000 if not pd.isna(row.get('BA_Lanes', np.nan)) else 1000
    
    # Get speed
    speed_ab = row.get('AB_FinalSpeed', np.nan) if not pd.isna(row.get('AB_FinalSpeed', np.nan)) else None
    speed_ba = row.get('BA_FinalSpeed', np.nan) if not pd.isna(row.get('BA_FinalSpeed', np.nan)) else None
    
    # Convert geometry to WKB for efficient insertion
    geom_wkb = wkb_dumps(geom, hex=False)
    
    # Create AB link (direction = 1 means one-way AB)
    if not pd.isna(travel_time_ab) and travel_time_ab > 0:
        # Use source_link_id for unidirectional, or generate unique ID for bidirectional
        if direction == 1:
            link_id = source_link_id
        else:
            link_id = source_link_id * 1000 + 1
        
        # Prepare tuple: (link_id, a_node, b_node, direction, distance, modes, link_type, 
        #                 travel_time_ab, travel_time_ba, capacity_ab, capacity_ba, 
        #                 speed_ab, speed_ba, geometry_wkb)
        link_data.append((
            link_id, a_node_id, b_node_id, 1, length, 'c', 'y',  # direction=1 (one-way AB)
            travel_time_ab, travel_time_ba if not pd.isna(travel_time_ba) else None,
            capacity_ab, capacity_ba,
            speed_ab, speed_ba,
            geom_wkb
        ))
        link_id_counter += 1
    
    # Create BA link if bidirectional (direction = 0)
    if direction == 0 and not pd.isna(travel_time_ba) and travel_time_ba > 0:
        link_id = source_link_id * 1000 + 2
        
        link_data.append((
            link_id, b_node_id, a_node_id, 1, length, 'c', 'y',  # direction=1 (one-way BA)
            travel_time_ba, travel_time_ab if not pd.isna(travel_time_ab) else None,
            capacity_ba, capacity_ab,
            speed_ba, speed_ab,
            geom_wkb
        ))
        link_id_counter += 1

print(f"Prepared {len(link_data)} links for insertion")

# Final validation: filter out links with invalid node references
validated_link_data = []
invalid_node_count = 0
for link_tuple in link_data:
    link_id = link_tuple[0]
    a_node = link_tuple[1]
    b_node = link_tuple[2]
    
    # Double-check that both nodes exist in database
    if a_node in db_nodes and b_node in db_nodes:
        validated_link_data.append(link_tuple)
    else:
        invalid_node_count += 1

if invalid_node_count > 0:
    print(f"Warning: Filtered out {invalid_node_count} links with invalid node references")

# Check for duplicate link_ids
seen_link_ids = set()
duplicate_links = []
unique_link_data = []
for link_tuple in validated_link_data:
    link_id = link_tuple[0]
    if link_id in seen_link_ids:
        duplicate_links.append(link_id)
    else:
        seen_link_ids.add(link_id)
        unique_link_data.append(link_tuple)

if duplicate_links:
    print(f"Warning: Found {len(duplicate_links)} duplicate link_ids, keeping first occurrence")
    print(f"Examples: {duplicate_links[:10]}")

print(f"Inserting {len(unique_link_data)} unique, validated links...")

# Bulk insert links
print("Bulk inserting links...")
with project.db_connection_spatial as conn:
    # Temporarily disable foreign key checks for bulk insert
    # We've already validated nodes exist, so this is safe
    conn.execute("PRAGMA foreign_keys = OFF")
    
    # Remove triggers for faster insertion
    remove_triggers(conn, project.logger, "network")
    
    # Insert links using GeomFromWKB for geometry
    # Use INSERT OR IGNORE to handle duplicate link_ids gracefully
    # Note: source_id field removed as it's not in the standard links table schema
    insert_qry = """INSERT OR IGNORE INTO links 
                    (link_id, a_node, b_node, direction, distance, modes, link_type,
                     travel_time_ab, travel_time_ba, capacity_ab, capacity_ba,
                     speed_ab, speed_ba, geometry)
                    VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, GeomFromWKB(?, 4326))"""
    
    # Insert in batches to handle large datasets better
    batch_size = 10000
    total_inserted = 0
    for i in range(0, len(unique_link_data), batch_size):
        batch = unique_link_data[i:i+batch_size]
        conn.executemany(insert_qry, batch)
        total_inserted += len(batch)
        if (i // batch_size + 1) % 10 == 0:
            print(f"  Inserted {total_inserted}/{len(unique_link_data)} links...")
    
    conn.commit()
    
    # Re-enable foreign key checks
    conn.execute("PRAGMA foreign_keys = ON")
    
    # Verify foreign key constraints are satisfied
    # Check for any links that reference non-existent nodes and remove them
    try:
        # Find links with invalid node references
        invalid_links = conn.execute("""
            SELECT l.link_id 
            FROM links l 
            LEFT JOIN nodes n1 ON l.a_node = n1.node_id 
            LEFT JOIN nodes n2 ON l.b_node = n2.node_id 
            WHERE n1.node_id IS NULL OR n2.node_id IS NULL
        """).fetchall()
        
        if invalid_links:
            print(f"Warning: Found {len(invalid_links)} links with invalid node references, removing...")
            for (link_id,) in invalid_links:
                conn.execute("DELETE FROM links WHERE link_id = ?", (link_id,))
            conn.commit()
    except Exception as e:
        print(f"Note: Could not verify foreign key constraints: {e}")
    
    # Check how many were actually inserted
    result = conn.execute("SELECT COUNT(*) FROM links")
    inserted_count = result.fetchone()[0]
    print(f"Total links in database: {inserted_count}")
    
    # Re-add triggers
    add_triggers(conn, project.logger, "network")
    conn.commit()

print(f"Successfully inserted {len(link_data)} links")

# Network is already up to date after bulk insert
# Verify network counts
print(f"\nNetwork summary:")
print(f"  Nodes: {project.network.count_nodes()}")
print(f"  Links: {project.network.count_links()}")
print(f"  Centroids: {project.network.count_centroids()}")
print("Network ready for use!")


Preparing links for bulk insertion...
Querying database for inserted nodes...
Found 348495 nodes in database
Using 348495 valid nodes for link matching
Prepared 652002 links for insertion
Examples: [10001, 10002, 19002, 20001, 20002, 41001, 41002, 51002, 52002, 265001]
Inserting 651881 unique, validated links...
Bulk inserting links...
  Inserted 100000/651881 links...
  Inserted 200000/651881 links...
  Inserted 300000/651881 links...
  Inserted 400000/651881 links...
  Inserted 500000/651881 links...
  Inserted 600000/651881 links...
Total links in database: 651881
Successfully inserted 652002 links


AttributeError: 'Network' object has no attribute 'refresh_network'

In [48]:
[name for name in dir(project.network) if not name.startswith("__")]

['_Network__count_items',
 'build_graphs',
 'convex_hull',
 'count_centroids',
 'count_links',
 'count_nodes',
 'create_from_gmns',
 'create_from_osm',
 'export_to_gmns',
 'extent',
 'graphs',
 'link_types',
 'links',
 'list_modes',
 'logger',
 'modes',
 'nodes',
 'periods',
 'project',
 'protected_fields',
 'req_link_flds',
 'req_node_flds',
 'set_time_field',
 'signal',
 'skimmable_fields']

## 3. Build Graphs

Build the graph structures needed for path computation and traffic assignment.


In [ ]:
# Build graphs
print("Building graphs...")
project.network.build_graphs()

# Get the graph for cars
graph = project.network.graphs["c"]
print(f"Available graphs: {list(project.network.graphs.keys())}")

# Set graph cost field to travel_time
graph.set_graph("travel_time")

# Set skimming fields
graph.set_skimming(["travel_time", "distance"])

# Get centroids
centroids = project.network.nodes.data[project.network.nodes.data.is_centroid == 1]
centroid_ids = centroids.node_id.values
print(f"Found {len(centroid_ids)} centroids")

# Prepare graph with centroids
graph.prepare_graph(centroid_ids)
print("Graph prepared successfully")


## 4. Calculate Shortest Travel Times/Costs

Calculate shortest travel times between random OD-pairs.


In [ ]:
# Select random OD-pairs from centroids
np.random.seed(42)  # For reproducibility
n_pairs = 10
random_indices = np.random.choice(len(centroid_ids), size=min(n_pairs * 2, len(centroid_ids)), replace=False)
od_pairs = [(centroid_ids[random_indices[i]], centroid_ids[random_indices[i+1]]) 
            for i in range(0, len(random_indices)-1, 2)][:n_pairs]

print(f"Selected {len(od_pairs)} OD-pairs for analysis")
print("OD-pairs:", od_pairs[:5], "...")


In [ ]:
# Calculate shortest path costs for each OD-pair
results = []
links_data = project.network.links.data.set_index('link_id')

for origin, destination in od_pairs:
    try:
        res = graph.compute_path(origin, destination)
        if res is not None and res.path is not None and len(res.path) > 0:
            # Get travel time from milepost (cumulative cost)
            travel_time = res.milepost[-1] if len(res.milepost) > 0 else np.nan
            
            # Calculate distance by summing link distances along the path
            path_links = res.path
            distance = 0.0
            for link_id in path_links:
                link_id_abs = abs(link_id)
                if link_id_abs in links_data.index:
                    link_dist = links_data.loc[link_id_abs, 'distance']
                    if pd.notna(link_dist):
                        distance += link_dist
            
            results.append({
                'origin': origin,
                'destination': destination,
                'travel_time': travel_time,
                'distance': distance,
                'path_length': len(res.path)
            })
    except Exception as e:
        print(f"Error computing path from {origin} to {destination}: {e}")
        continue

# Create results dataframe
shortest_costs_df = pd.DataFrame(results)
print(f"\nComputed shortest paths for {len(shortest_costs_df)} OD-pairs")
shortest_costs_df.head(10)


## 5. Compute Shortest Paths

Compute detailed shortest paths between OD-pairs and extract path information.


In [ ]:
# Compute detailed shortest paths
path_results = []
links_data = project.network.links.data.set_index('link_id')

# Select a few OD-pairs for detailed path analysis
selected_od_pairs = od_pairs[:5]

for origin, destination in selected_od_pairs:
    try:
        res = graph.compute_path(origin, destination)
        if res is not None and res.path is not None and len(res.path) > 0:
            # Get travel time from milepost
            travel_time = res.milepost[-1] if len(res.milepost) > 0 else np.nan
            
            # Calculate distance by summing link distances along the path
            path_links = res.path
            distance = 0.0
            for link_id in path_links:
                link_id_abs = abs(link_id)
                if link_id_abs in links_data.index:
                    link_dist = links_data.loc[link_id_abs, 'distance']
                    if pd.notna(link_dist):
                        distance += link_dist
            
            path_info = {
                'origin': origin,
                'destination': destination,
                'path_nodes': res.path_nodes.tolist() if hasattr(res, 'path_nodes') else [],
                'path_links': res.path.tolist(),
                'travel_time': travel_time,
                'distance': distance,
                'num_nodes': len(res.path_nodes) if hasattr(res, 'path_nodes') else 0,
                'num_links': len(res.path)
            }
            path_results.append(path_info)
            print(f"Path from {origin} to {destination}: {path_info['num_links']} links, "
                  f"travel time: {path_info['travel_time']:.2f}, distance: {path_info['distance']:.2f}")
    except Exception as e:
        print(f"Error computing path from {origin} to {destination}: {e}")
        continue

paths_df = pd.DataFrame(path_results)
print(f"\nComputed detailed paths for {len(paths_df)} OD-pairs")
paths_df[['origin', 'destination', 'num_links', 'travel_time', 'distance']]


## 6. Generate Route Choice Sets

Generate route choice sets between OD pairs using the RouteChoice class.


In [ ]:
# Initialize RouteChoice
print("Initializing RouteChoice...")
rc = RouteChoice(graph)

# Set choice set generation parameters
# Using BFS-LE (Breadth-First Search with Link Elimination) algorithm
rc.set_choice_set_generation("bfsle", max_routes=5, penalty=1.05)

# Select a smaller subset of OD-pairs for route choice (this can be computationally intensive)
route_choice_od_pairs = selected_od_pairs[:3]
print(f"Generating route choice sets for {len(route_choice_od_pairs)} OD-pairs...")

# Prepare and execute route choice
rc.prepare(route_choice_od_pairs)
rc.execute(perform_assignment=False)

# Get results
choice_set_results = rc.get_results()
print(f"\nGenerated route choice sets:")
print(choice_set_results.head())


In [ ]:
# Display route choice set details
if choice_set_results is not None and len(choice_set_results) > 0:
    print("Route Choice Set Summary:")
    print(f"Total OD-pairs: {len(choice_set_results)}")
    
    # Show number of routes per OD-pair
    if 'route set' in choice_set_results.columns:
        route_counts = choice_set_results.apply(lambda x: len(x['route set']) if isinstance(x['route set'], (list, tuple)) else 0, axis=1)
        print(f"\nRoutes per OD-pair:")
        print(route_counts.describe())
    
    # Display first few results
    print("\nFirst few route choice sets:")
    display_cols = [col for col in choice_set_results.columns if col != 'route set']
    print(choice_set_results[display_cols].head())
else:
    print("No route choice sets generated")


## 7. Traffic Assignment with Path-Sized Logit

Perform traffic assignment using the path-sized logit method.


In [ ]:
# Create a demand matrix for traffic assignment
# For demonstration, we'll create a simple random demand matrix
# In practice, you would load actual OD demand data

print("Creating demand matrix...")

# Get number of zones (centroids)
num_zones = len(centroid_ids)
zone_index = centroid_ids

# Create a simple demand matrix (random values for demonstration)
# In practice, load actual demand data
np.random.seed(42)
demand_matrix = np.random.rand(num_zones, num_zones) * 1000
# Set diagonal to zero (no internal trips)
np.fill_diagonal(demand_matrix, 0)

# Create AequilibraeMatrix
matrix_file = join(project_folder, "demand.aem")
aem = AequilibraeMatrix()
kwargs = {
    'file_name': matrix_file,
    'zones': num_zones,
    'matrix_names': ['demand']
}
aem.create_empty(**kwargs)
aem.matrix['demand'][:, :] = demand_matrix[:, :]
aem.index[:] = zone_index[:]

# Set computational view
aem.computational_view(['demand'])
print(f"Created demand matrix with {num_zones} zones")
print(f"Total demand: {demand_matrix.sum():.0f} trips")


In [ ]:
# Set up traffic assignment with path-sized logit using RouteChoice
print("Setting up route choice assignment with path-sized logit...")

# Create RouteChoice object
rc_assignment = RouteChoice(graph)

# Add demand matrix to route choice
rc_assignment.add_demand(aem)

# Set choice set generation parameters
rc_assignment.set_choice_set_generation("bfsle", max_routes=5, penalty=1.05)

# Set path-sized logit parameters
rc_assignment.set_path_size_logit(beta_psl=1.0, min_share=0.01)

# Get OD pairs from demand matrix for preparation
od_list = []
for i, orig in enumerate(centroid_ids):
    for j, dest in enumerate(centroid_ids):
        if demand_matrix[i, j] > 0:
            od_list.append((int(orig), int(dest)))

print(f"Prepared route choice for {len(od_list)} OD pairs with demand")
print("Route choice configured with path-sized logit")


In [ ]:
# Execute route choice assignment with path-sized logit
print("Executing route choice assignment with path-sized logit...")
print("This may take several minutes depending on network size and number of OD pairs...")

try:
    # Prepare route choice with OD pairs
    rc_assignment.prepare(od_list)
    
    # Execute route choice with assignment (path-sized logit)
    rc_assignment.execute(perform_assignment=True)
    
    print("\nRoute choice assignment completed successfully!")
    
    # Get results
    choice_results = rc_assignment.get_results()
    
    print("\nRoute Choice Assignment Results Summary:")
    print(f"Total OD-pairs processed: {len(choice_results) if choice_results is not None else 0}")
    
    # Get link loads from route choice
    if hasattr(rc_assignment, 'link_loads'):
        link_volumes = rc_assignment.link_loads
        print(f"\nLink volumes computed for {len(link_volumes)} links")
        print(f"Total assigned volume: {link_volumes.sum():.0f}")
        print(f"Average link volume: {link_volumes.mean():.2f}")
        print(f"Maximum link volume: {link_volumes.max():.2f}")
    else:
        print("\nNote: Link volumes are stored in the route choice results")
        print("Access via rc_assignment.get_results() for detailed route-level assignments")
    
except Exception as e:
    print(f"Error during route choice assignment: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Display link volumes on network
if 'link_volumes' in locals() and link_volumes is not None:
    # Get link data
    links_data = project.network.links.data
    
    # Add volumes to links dataframe
    links_with_volumes = links_data.copy()
    links_with_volumes['volume'] = 0.0
    
    # Map volumes to links (this is a simplified mapping)
    # In practice, you would use the proper link_id mapping from assignment results
    print("Link volumes summary:")
    print(f"Links with volume > 0: {(link_volumes > 0).sum()}")
    print(f"Top 10 link volumes:")
    
    # Get top volumes
    if len(link_volumes) > 0:
        top_volumes = pd.Series(link_volumes).nlargest(10)
        print(top_volumes)


## Summary

This notebook demonstrated:
1. ✅ Loading FAF5 network from Geodatabase format
2. ✅ Creating AequilibraE project and importing network
3. ✅ Calculating shortest travel times/costs between OD-pairs
4. ✅ Computing shortest paths with detailed information
5. ✅ Generating route choice sets using BFS-LE algorithm
6. ✅ Performing traffic assignment with path-sized logit method

The analysis is complete. You can now explore the results further or modify parameters for different scenarios.


In [ ]:
# Clean up - close the project
project.close()
print("Project closed")
